In [1]:
!pip install dash 
!pip install dash-html-components 
!pip install dash-core-components 
!pip install dash-table 

In [2]:
!pip install pandas
!pip install numpy

In [3]:
import pandas as pd
import numpy as np

In [68]:
import dash
import pandas as pd
from dash import dcc, html, Input, Output, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

def combine_names(row):
    return f"{row['Фамилия']} {row['Имя']} {row['Отчество']}"

g_persons_df = pd.read_csv('./db_persons.csv')[['ID', 'Фамилия', 'Имя', 'Отчество']]
g_persons_df['ФИО'] = g_persons_df.apply(combine_names, axis=1)
gpersons = g_persons_df[['ФИО', 'ID']]
gpersons.columns = ['label', 'value']
gpersons = gpersons.to_dict(orient='records')

app.layout = html.Div([
    html.Label('Выберите автора'),
    dcc.Dropdown(
        options=gpersons,
        placeholder="Выберите автора",
        id='author',
    ),
    html.Div(id='selected-author'),  # Для вывода выбранного значения
    dcc.Dropdown(id='works-dropdown', 
                 multi=True,
                 optionHeight=40)  # Для списка "Работы"
], style={'columnCount': 1})

# Глобальная переменная для хранения значения автора
selected_author_value = None

# Функция обратного вызова для обновления выбранного автора
@app.callback(
    Output('selected-author', 'children'),
    Input('author', 'value')
)
def update_selected_author(selected_value):
    global selected_author_value
    selected_author_value = selected_value
    
    if selected_value is None:
        return "Выберите автора"
    else:
        return f"Выбран автор: {selected_value}"

# Функция обратного вызова для обновления списка "Работы"
@app.callback(
    Output('works-dropdown', 'options'),
    Input('author', 'value')
)
def update_works_dropdown(selected_value):
    if selected_value is None:
        return []

    # Загрузка данных из файла "data_from_json_files.csv"
    data_df = pd.read_csv('./data_from_json_files.csv')

    # Преобразование выбранного ID автора в строку
    selected_value_str = str(selected_value)

    # Разделение значения "Авторы" на список ID авторов
    data_df['Авторы'] = data_df['Авторы'].apply(lambda x: [] if pd.isna(x) else x.split(', '))

    # Фильтрация строк, в которых в списке ID авторов есть выбранный автор
    works = data_df[data_df['Авторы'].apply(lambda x: selected_value_str in x)]

    works_data = works[['Название', 'ID']].drop_duplicates(subset=['ID'])
    
    # Обрезаем названия работ до первых 15 символов
    works_data['label'] = works_data['Название'].str[:25]

    # Преобразуем опции в список словарей с ключами "label" и "value"
    options = [{'label': work['label'], 'value': work['ID']} for _, work in works_data.iterrows()]

    return options



if __name__ == '__main__':
    app.run_server(debug=True)
